# HW5

For this homework, we are going to work with [*Indoor User Movement Prediction from RSS data*](https://archive.ics.uci.edu/ml/datasets/Indoor+User+Movement+Prediction+from+RSS+data) dataset from UCI.  The homework is due Friday, December 21st midnight. 

## Task 1

Download the dataset and unzip it in under a subdirectory of `data` folder named `rss_data`.

The files we are interested is in the subfolder `dataset`.  Each of these files whose names that start with `MovementAAL_RSS_` contain data collected from indivuduals. Each of these files represent a single data point.  There are 314 of these files, and hence, you have 314 data points.  Each file has 4 columns but the number of rows change from file to file.  

There is also a file named `MovementALL_target.csv` in that folder. This file tells us the class each of these measurements are assigned. Some of these measurements are labelled with +1 and some are labelled with -1.

## Task 2

Construct a SVM model that separates +1 labelled data points from -1 data points.  You must first solve the problem that these datapoints do not have the same number of rows even though they all have the same number of columns. 

## Task 3

Using [Keras](https://keras.io/getting-started/sequential-model-guide/) write a neural network model that separates +1 labelled data points from -1 data points.

## Notes

1. You must document each step of your tasks: what are you doing, why are you doing it, what problems you encountered and how you solved it.  All of these must be explained and documented.  Solutions without sufficient documentations will be penalized accordingly. 50% of your points will come from your code, while the other 50% will come from your explanations.

1. You can use MS Excel to inspect the files, but loading them up to python using pandas and inspecting them there under jupyter is easier.

3. Put the data in a separate subfolder of your `data` folder and rename it `rss_data`. I'll take points off if the data is not saved under the correct place.

1. For both of Task 2 and Task 3, you must split your data into a train and test set, and then evaluate the accuracy of your model on the test set.




# TASK 1:

Data yı hazırlıyoruz

In [1]:
import glob
import pandas as pd
import numpy as np

Gerekli kütüphaneler yüklendi

In [2]:
csv_files=glob.glob("..//data//rss_data//dataset//MovementAAL_RSS_*.csv")
frames=[]
for csv in csv_files:
    df=pd.read_csv(csv)
    frames.append(df)

glob kütüphanesinin glob fonksiyonuyla 314 adet datayı csv_files listesine atadık for döngüsüyle de frames listesine attık

In [3]:
a=100
for i in frames:
    a=min(a,len(i))
print(a)

19


Datalar eşit değil en düşük olan sayıdakini bulup sondan kesicez son bulunduğu lokasyonu tespit için

In [4]:
l=[]
for i in frames:
    for row in range(1,20):
        for col in range(4):
            l.append(i.iloc[-row,col])

tüm datayı düz liste yapıp tekrar şekillendirip dataframe haline getiriyorum

In [5]:
k=np.array(l)

In [6]:
k=k.reshape(314,76)

In [7]:
df=pd.DataFrame(k)

target datasını çekiyorum hangi odada fln olduğunu gösteren data ,(314,2) geldiği için sequence kolonunu atıyorum

In [8]:
targets = pd.read_csv('..//data//rss_data//dataset//MovementAAL_target.csv')
targets=targets.drop("#sequence_ID",axis=1)

In [9]:
df=np.column_stack([df,targets])

In [10]:
df=pd.DataFrame(df)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,-0.14286,-0.60,-0.28571,-0.10,0.00000,0.04,-0.047619,-0.05,0.00000,0.04,...,0.40,-0.047619,0.04,-0.095238,0.05,-0.095238,-0.08,0.142860,-0.55,1.0
1,0.19048,0.04,-0.38095,0.20,0.19048,0.00,-0.190480,-0.25,0.14286,-0.04,...,0.10,0.000000,-0.12,0.142860,0.10,-0.285710,-0.24,0.095238,0.15,1.0
2,0.28571,0.56,-0.57143,-0.75,0.33333,0.40,-0.952380,-0.65,0.14286,0.56,...,-0.70,0.428570,0.04,-0.619050,-0.70,0.047619,0.32,-0.714290,-0.30,1.0
3,0.28571,0.20,-1.00000,-0.70,0.00000,0.04,-1.000000,-0.55,0.00000,0.04,...,-0.65,0.523810,0.60,-0.428570,-0.55,0.476190,0.32,-0.380950,-0.30,1.0
4,0.42857,0.68,-0.28571,-0.30,0.42857,0.76,-0.476190,-0.60,0.00000,0.36,...,-1.00,0.285710,0.60,-0.571430,-1.00,0.142860,0.76,-0.666670,-0.55,1.0


dataframe haline getirilirdi

data hazırlandı.

# Task 2:

Svm kütüphanelerini alma datayı bölme işlemleri aşağıda rfb en etkili olduğundan rfb yi seçtim gamma bir çok deneme sonrası optimal burda kaldı

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score


In [12]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,:76],df.iloc[:,76])
sv=SVC(kernel="rbf",gamma=0.03,C=2)
sv.fit(X_train,y_train)
predicted=sv.predict(X_test)
print(accuracy_score(y_test,predicted))
print(confusion_matrix(y_test,predicted))

0.7341772151898734
[[28 12]
 [ 9 30]]


data sonucu yeterli değil neural network te daha başarılı olucak muhtemelen

# TASK 3:

Keras kütüphanelerini atma

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM,Dropout
from keras.preprocessing import sequence
from keras.utils import to_categorical

Using TensorFlow backend.


Datayı uygun hale getirme 

In [14]:
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
X_train = X_train.as_matrix()
X_test = X_test.as_matrix()

C:\Users\gurbu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\gurbu\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


classification da categoric hale getirdik targeti.predictor ı da array haline getirdik

In [15]:
model= Sequential()
model.add(Dense(100,activation="relu",input_shape=(76,)))
model.add(Dense(10,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10,activation="relu"))
model.add(Dense(10,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10,activation="relu"))
model.add(Dense(10,activation="relu"))
model.add(Dense(10,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10,activation="relu"))
model.add(Dense(10,activation="relu"))
model.add(Dense(2,activation="softmax"))
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test,y_test))

Train on 235 samples, validate on 79 samples
Epoch 1/10
235/235 [==============================] - 1s 3ms/step - loss: 0.6792 - acc: 0.8936 - val_loss: 0.6618 - val_acc: 1.0000
Epoch 2/10
235/235 [==============================] - 0s 66us/step - loss: 0.6332 - acc: 1.0000 - val_loss: 0.6182 - val_acc: 1.0000
Epoch 3/10
235/235 [==============================] - 0s 133us/step - loss: 0.5736 - acc: 1.0000 - val_loss: 0.5421 - val_acc: 1.0000
Epoch 4/10
235/235 [==============================] - 0s 66us/step - loss: 0.4876 - acc: 1.0000 - val_loss: 0.4229 - val_acc: 1.0000
Epoch 5/10
235/235 [==============================] - 0s 66us/step - loss: 0.3684 - acc: 1.0000 - val_loss: 0.2743 - val_acc: 1.0000
Epoch 6/10
235/235 [==============================] - 0s 66us/step - loss: 0.3311 - acc: 1.0000 - val_loss: 0.1552 - val_acc: 1.0000
Epoch 7/10
235/235 [==============================] - 0s 67us/step - loss: 0.2830 - acc: 1.0000 - val_loss: 0.0841 - val_acc: 1.0000
Epoch 8/10
235/235 [====

it shows us neural networks very good 

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               7700      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                110       
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                110       
__________